## _*Using Qiskit Aqua algorithms, a how to guide*_

This notebook demonstrates how to use the `Qiskit Aqua` library to invoke an algorithm and process the result.

Further information may be found for the algorithms in the online [Aqua documentation](https://qiskit.org/documentation/apidoc/qiskit.aqua.algorithms.html).

Algorithms in Aqua can be created and run as usual in Python by constructing instances and calling methods.

Aqua has many `algorithms` for solving different problems. For some we also have classical algorithms, that take the exact same input data, to solve the problem. This can be useful in the near term as Quantum algorithms are developed since we are still at a stage where we can do classical comparison of the result.

Aqua also has various `components` which are dependent objects used by algorithms, such as variational forms, qfts, initial states etc. We will see more on this below.

Lastly for developers we also have a collections of `circuits` and gates which can be used to help build out new components and algorithms.

Here we will choose to show some of the main aspects of Aqua by solving a ground state energy problem.

In [1]:
from qiskit.aqua.operators import WeightedPauliOperator

As input, for an energy problem, we need a Hamiltonian and so we first create a suitable `Operator ` instance. In this case we have a paulis list, as below, from a previously computed Hamiltonian, that we saved, so as to focus this notebook on using the algorithms. We simply load these paulis to create the original Operator.

This Hamiltonian was created originally using Qiskit Chemistry for an H2 molecule at 0.735A interatomic distance. Please refer to the chemistry tutorials here if you are interested in understanding more. Suffice to say at this level Aqua does not really care about the source of the Operator.

In [2]:
pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "ZI"},
              {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "IZ"},
              {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
}

qubit_op = WeightedPauliOperator.from_dict(pauli_dict)

### Let's start with a classical algorithm

We can now use the Operator without regard to how it was created. We chose to start this tutorial with a classical algorithm as it involves a little less setting up than the `VQE` quantum algorithm we will use later. Here we will use `NumPyMinimumEigensolver` to compute the minimum eigenvalue of the Operator (Hamiltonian).

We construct an `NumPyMinimumEigensolver` instance, passing in the Operator, and then call `run()` on in order to compute the result. All Aqua algorithms have the run method (it is defined by a base class which all algorithms extend) and while no parameters are need for classical algorithms a quantum algorithm will require a backend (quantum simulator or real device) on which it will be run. The `result` object returned is a dictionary. While the results fields can be different for algorithms solving different problems, and even within a given problem type there may be algorithm specific data returned, for a given problem the fields core to that problem are common across algorithms in order that different algorithms can be chosen to solve the same problem in a consistent fashion.

In [3]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver

ee = NumPyMinimumEigensolver(qubit_op)
result = ee.run()
print(result.eigenvalue.real)

-1.8572750302023797


### Lets switch now to using a Quantum algorithm.

We will use the Variational Quantum Eigensolver (VQE) to solve the same problem as above. As its name implies its uses a variational approach. An ansatz (a variational form) is supplied and using a quantum/classical hybrid technique the energy resulting from evaluating the Operator with the variational form on a quantum backend is taken down to a minimum using a classical optimizer that varies the parameters of the variational form.

Here we create the variational form and optimizer and then pass them to VQE along with the Operator. The backend is created and passed to the algorithm so it can be run there.

In [4]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.components.optimizers import L_BFGS_B

var_form = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cz', reps=3, entanglement='linear')
optimizer = L_BFGS_B(maxfun=1000)
vqe = VQE(qubit_op, var_form, optimizer)
backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)
print(result.eigenvalue.real)

-1.85727503014758


The following shows the above but using a QuantumInstance.

In [5]:
from qiskit.aqua import QuantumInstance

var_form = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cz', reps=3, entanglement='linear')
optimizer = L_BFGS_B(maxfun=1000)
vqe = VQE(qubit_op, var_form, optimizer)
backend = BasicAer.get_backend('statevector_simulator')
qi = QuantumInstance(backend=backend)
result = vqe.run(qi)
print(result.eigenvalue.real)

-1.85727503011329


### Concluding

This completes an introduction to programming and using Aqua algorithms. There are plenty of other  tutorials showing Aqua being used to solve other problems, including Machine Learning, Finance, Optimization and Chemistry. We encourage you to explore these further and see that various capabilities and techniques employed.